In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
# Define the URL for the SEC EDGAR search page
url = "https://www.sec.gov/files/company_tickers_exchange.json"

headers = {
   "User-Agent": "your.email@email.com"
}
# Fetch the HTML content of the search page
response = requests.get(url,headers=headers)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
# Parse the JSON content from the BeautifulSoup object
json_data = json.loads(soup.text)

# Convert the JSON data into a pandas DataFrame
df_cik_lookup = pd.DataFrame(json_data['data'], columns=json_data['fields'])


# df_cik_lookup['cik']=df_cik_lookup['cik'].astype(str).zfill(10)
df_cik_lookup

,cik,name,ticker,exchange
0,320193,Apple Inc.,AAPL,Nasdaq
1,1045810,NVIDIA CORP,NVDA,Nasdaq
2,789019,MICROSOFT CORP,MSFT,Nasdaq
3,1018724,AMAZON COM INC,AMZN,Nasdaq
4,1652044,Alphabet Inc.,GOOGL,Nasdaq
...,...,...,...,...
10003,2029769,Oaktree Acquisition Corp. III Life Sciences,OACCU,Nasdaq
10004,2031561,Aldel Financial II Inc.,ALDFU,Nasdaq
10005,2032379,Willow Lane Acquisition Corp.,WLACU,Nasdaq
10006,2032950,Horizon Space Acquisition II Corp.,HSPTU,Nasdaq


In [ ]:
df_cik_lookup.head()

,cik,name,ticker,exchange
0,320193,Apple Inc.,AAPL,Nasdaq
1,1045810,NVIDIA CORP,NVDA,Nasdaq
2,789019,MICROSOFT CORP,MSFT,Nasdaq
3,1018724,AMAZON COM INC,AMZN,Nasdaq
4,1652044,Alphabet Inc.,GOOGL,Nasdaq


Get a list of companies listed in NASDAQ, filter for Technology companies with an active IPO year, market cap > 1 Mil, Country is USA and volume of shares is greater than 1000

In [ ]:

# Load the CSV data into a pandas DataFrame
df = pd.read_csv('nasdaq_screener_1729013554057.csv')

# Convert the 'IPO Year' column to integer format, handling potential errors
df['IPO Year'] = pd.to_numeric(df['IPO Year'], errors='coerce').astype('Int64')

sectors_to_consider = ['Technology', 'Consumer Discretionary', 'Industrials', 'Finance', 'Health Care']

# Filter the DataFrame based on the specified criteria
filtered_df = df[(df['Country'] == 'United States') &
                 (df['Sector'].isin(sectors_to_consider)) &
                 (df['IPO Year'].notna()) &
                 (df['Market Cap'] > 1000000) &
                 (df['Volume'] >= 1000)]

filtered_df = filtered_df[~filtered_df['Name'].str.contains('warrant', case=False)]

# Sort the DataFrame in descending order based on 'Market Cap' and select the top 50
filtered_df = filtered_df.sort_values('Market Cap', ascending=False).head(500)


columns_to_remove = ['Last Sale', 'Net Change', 'Unnamed: 8', '% Change', 'Volume']

# Remove the specified columns from the DataFrame
companies_considered = filtered_df.drop(columns=columns_to_remove, errors='ignore')

# assign a new index starting from 1
companies_considered.index = np.arange(1, len(companies_considered) + 1)

print(companies_considered)

    Symbol                                               Name    Market Cap  \
1     AAPL                            Apple Inc. Common Stock  3.563394e+12   
2     NVDA                    NVIDIA Corporation Common Stock  3.220298e+12   
3     MSFT                 Microsoft Corporation Common Stock  3.105449e+12   
4     GOOG                Alphabet Inc. Class C Capital Stock  2.062048e+12   
5    GOOGL                 Alphabet Inc. Class A Common Stock  2.044937e+12   
..     ...                                                ...           ...   
496   TFSL             TFS Financial Corporation Common Stock  3.725033e+09   
497   ASBA  Associated Banc-Corp 6.625% Fixed-Rate Reset S...  3.722723e+09   
498     ZI             ZoomInfo Technologies Inc Common Stock  3.722203e+09   
499    GMS                              GMS Inc. Common Stock  3.720706e+09   
500    SPR      Spirit Aerosystems Holdings Inc. Common Stock  3.700366e+09   

           Country  IPO Year                  Secto

In [ ]:

# Perform an inner join on 'ticker' and 'Symbol' columns
merged_df = pd.merge(df_cik_lookup, companies_considered, left_on='ticker', right_on='Symbol', how='inner')

# Print the merged DataFrame
print(merged_df.columns)

print(merged_df)

Index(['cik', 'name', 'ticker', 'exchange', 'Symbol', 'Name', 'Market Cap',
       'Country', 'IPO Year', 'Sector', 'Industry'],
      dtype='object')
         cik                                name ticker exchange Symbol  \
0     320193                          Apple Inc.   AAPL   Nasdaq   AAPL   
1    1045810                         NVIDIA CORP   NVDA   Nasdaq   NVDA   
2     789019                      MICROSOFT CORP   MSFT   Nasdaq   MSFT   
3    1018724                      AMAZON COM INC   AMZN   Nasdaq   AMZN   
4    1652044                       Alphabet Inc.  GOOGL   Nasdaq  GOOGL   
..       ...                                 ...    ...      ...    ...   
493  1137774            PRUDENTIAL FINANCIAL INC    PFH     NYSE    PFH   
494  1137774            PRUDENTIAL FINANCIAL INC    PRH     NYSE    PRH   
495  1137774            PRUDENTIAL FINANCIAL INC    PRS     NYSE    PRS   
496  1858681      Apollo Global Management, Inc.   APOS     NYSE   APOS   
497  1865782  BrightSpri

In [ ]:

sector_df = merged_df[['cik', 'Sector', 'Industry']].drop_duplicates()
sector_df

,cik,Sector,Industry
0,320193,Technology,Computer Manufacturing
1,1045810,Technology,Semiconductors
2,789019,Technology,Computer Software: Prepackaged Software
3,1018724,Consumer Discretionary,Catalog/Specialty Distribution
4,1652044,Technology,Computer Software: Programming Data Processing
...,...,...,...
490,5513,Finance,Accident &Health Insurance
491,1336917,Consumer Discretionary,Apparel
492,7789,Finance,Major Banks
496,1858681,Finance,Investment Managers


In [ ]:
def make_edgar_request(url):
    """
    Make a request to EDGAR (Electronic Data Gathering, Analysis and Retrieval)
    :param url: request URL
    :return: response
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br",
    }
    return requests.get(url, headers=headers)

In [ ]:
def get_company_data(cik):
  """Fetches and processes data for a given CIK."""

  url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{str(cik).zfill(10)}.json"
  header = {
    "User-Agent": "your.email@email.com"  # Replace with your actual email
  }

  response = requests.get(url, headers=header)

  if response.status_code == 200:
    data = response.json()

    us_gaap_data = data.get("facts", {}).get("us-gaap", {})
    table_data = []

    for concept, values in us_gaap_data.items():
      for value in values.get("units", {}).get("USD", []):
        if "val" in value:
          table_data.append([concept, value.get("val"), value.get("fy"), value.get("fp")])

    df = pd.DataFrame(table_data, columns=["Concept", "Value", "Fiscal Year", "Fiscal Period"])
    entity_name = data.get("entityName", "Unknown")  # Get the entity name
    sector = data.get("Sector", "Unknown")  # Get the sector
    industry = data.get("Industry", "Unknown")  # Get the industry
    df["entityName"] = entity_name  # Add entity name as a column
    df["cik"] = cik  # Add CIK as a column
    return df
  else:
    print(f"Error: {response.status_code}")
    return None

In [ ]:
data_list = [
    "Assets",
    "CostsAndExpenses",
    "GrossProfit",
    "IncomeTaxExpenseBenefit",
    "Liabilities",
    "LiabilitiesAndStockholdersEquity",
    "NetIncomeLoss",
    "OperatingExpenses",
    "ProfitLoss",
    "PropertyPlantAndEquipmentGross",
    "ResearchAndDevelopmentExpense",
    "StockholdersEquity"
]

In [ ]:
# Get a list of CIKs from the merged DataFrame
CIKs_to_process = merged_df["cik"].tolist()

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

for CIK in CIKs_to_process:
  company_df = get_company_data(CIK)
  if company_df is not None:
    combined_df = pd.concat([combined_df, company_df], ignore_index=True)


# Perform an inner join on 'cik'
combined_df = pd.merge(sector_df, combined_df, on='cik', how='inner')
combined_df

combined_df.head()

<ipython-input-90-4ef76838a50a>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, company_df], ignore_index=True)
<ipython-input-90-4ef76838a50a>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, company_df], ignore_index=True)
<ipython-input-90-4ef76838a50a>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all

Error: 404


,cik,Sector,Industry,Concept,Value,Fiscal Year,Fiscal Period,entityName
0,320193,Technology,Computer Manufacturing,AccountsPayable,5.520000e+09,2009.0,Q3,Apple Inc.
1,320193,Technology,Computer Manufacturing,AccountsPayable,4.854000e+09,2009.0,Q3,Apple Inc.
2,320193,Technology,Computer Manufacturing,AccountsPayableCurrent,5.520000e+09,2009.0,FY,Apple Inc.
3,320193,Technology,Computer Manufacturing,AccountsPayableCurrent,5.520000e+09,2009.0,FY,Apple Inc.
4,320193,Technology,Computer Manufacturing,AccountsPayableCurrent,5.601000e+09,2009.0,FY,Apple Inc.


In [91]:
df_concepts = pd.DataFrame(data_list, columns=['Concept'])
df_concepts

,Concept
0,Assets
1,CostsAndExpenses
2,GrossProfit
3,IncomeTaxExpenseBenefit
4,Liabilities
5,LiabilitiesAndStockholdersEquity
6,NetIncomeLoss
7,OperatingExpenses
8,ProfitLoss
9,PropertyPlantAndEquipmentGross


In [92]:
# Perform a join on 'concepts' and 'Concept' columns
joined_df = pd.merge(df_concepts, combined_df, left_on='Concept', right_on='Concept', how='inner')

print(joined_df)

                   Concept      cik       Sector                  Industry  \
0                   Assets   320193   Technology    Computer Manufacturing   
1                   Assets   320193   Technology    Computer Manufacturing   
2                   Assets   320193   Technology    Computer Manufacturing   
3                   Assets   320193   Technology    Computer Manufacturing   
4                   Assets   320193   Technology    Computer Manufacturing   
...                    ...      ...          ...                       ...   
496974  StockholdersEquity  1865782  Health Care  Medical/Nursing Services   
496975  StockholdersEquity  1865782  Health Care  Medical/Nursing Services   
496976  StockholdersEquity  1865782  Health Care  Medical/Nursing Services   
496977  StockholdersEquity  1865782  Health Care  Medical/Nursing Services   
496978  StockholdersEquity  1865782  Health Care  Medical/Nursing Services   

               Value  Fiscal Year Fiscal Period  \
0       3.95

In [93]:

# Filter for Fiscal Period = FY
filtered_joined_df = joined_df[joined_df['Fiscal Period'] == 'FY']

# Make the data unique based on concept, fiscal year, and entityName
unique_df = filtered_joined_df.drop_duplicates(subset=['Concept', 'Fiscal Year', 'entityName'])

print(unique_df)

                   Concept      cik      Sector                Industry  \
1                   Assets   320193  Technology  Computer Manufacturing   
3                   Assets   320193  Technology  Computer Manufacturing   
11                  Assets   320193  Technology  Computer Manufacturing   
27                  Assets   320193  Technology  Computer Manufacturing   
37                  Assets   320193  Technology  Computer Manufacturing   
...                    ...      ...         ...                     ...   
496843  StockholdersEquity     7789     Finance             Major Banks   
496865  StockholdersEquity     7789     Finance             Major Banks   
496887  StockholdersEquity     7789     Finance             Major Banks   
496954  StockholdersEquity  1858681     Finance     Investment Managers   
496959  StockholdersEquity  1858681     Finance     Investment Managers   

               Value  Fiscal Year Fiscal Period  \
1       3.957200e+10       2009.0            FY 

In [123]:

#'unique_df' is the DataFrame we want to transpose
transposed_df = unique_df.pivot_table(index=['cik', 'entityName', 'Fiscal Year', 'Sector', 'Industry'], columns='Concept', values='Value', aggfunc='first')

# Reset the index to make 'cik', 'entityName', and 'Fiscal Year' regular columns
transposed_df = transposed_df.reset_index()

print(transposed_df)

Concept      cik                entityName  Fiscal Year  \
0           4457    U-Haul Holding Company       2011.0   
1           4457    U-Haul Holding Company       2013.0   
2           4457    U-Haul Holding Company       2014.0   
3           4457    U-Haul Holding Company       2015.0   
4           4457    U-Haul Holding Company       2016.0   
...          ...                       ...          ...   
4681     1955520   Knife River Corporation       2023.0   
4682     1957132          SHARKNINJA, INC.       2023.0   
4683     1967680       VERALTO CORPORATION       2023.0   
4684     1973266  TKO GROUP HOLDINGS, INC.       2023.0   
4685     1977303      ARCADIUM LITHIUM PLC       2023.0   

Concept                  Sector  \
0        Consumer Discretionary   
1        Consumer Discretionary   
2        Consumer Discretionary   
3        Consumer Discretionary   
4        Consumer Discretionary   
...                         ...   
4681                Industrials   
4682     Co

Descriptive Statistics: Use pandas to get a summary of the dataset, such as mean, median, standard deviation, and range:

In [124]:
# Get descriptive statistics
descriptive_stats = transposed_df.describe()

# Display the statistics
print(descriptive_stats)

Concept           cik  Fiscal Year        Assets  CostsAndExpenses  \
count    4.686000e+03  4686.000000  4.686000e+03      1.181000e+03   
mean     1.175474e+06  2017.709560  2.291203e+10      9.198851e+09   
std      4.187648e+05     3.978215  9.236596e+10      2.708708e+10   
min      4.457000e+03  2009.000000  5.102000e+03     -8.089220e+08   
25%      9.105210e+05  2015.000000  1.327291e+09      9.104000e+08   
50%      1.176948e+06  2018.000000  3.398314e+09      2.411900e+09   
75%      1.512673e+06  2021.000000  9.201750e+09      6.794639e+09   
max      1.977303e+06  2025.000000  1.463988e+12      4.449430e+11   

Concept   GrossProfit  IncomeTaxExpenseBenefit   Liabilities  \
count    2.626000e+03             4.571000e+03  3.869000e+03   
mean     2.096393e+09             2.215838e+08  1.994861e+10   
std      8.865091e+09             1.205646e+09  9.092611e+10   
min     -4.650000e+08            -3.483100e+10  1.276000e+03   
25%      2.785255e+08             2.864500e+06  5

Check for Missing Values: To find any missing values:

In [125]:
# Check for missing values
missing_values = transposed_df.isnull().sum()

# Display missing values for each column
print(missing_values)

Concept
cik                                    0
entityName                             0
Fiscal Year                            0
Sector                                 0
Industry                               0
Assets                                 0
CostsAndExpenses                    3505
GrossProfit                         2060
IncomeTaxExpenseBenefit              115
Liabilities                          817
LiabilitiesAndStockholdersEquity       0
NetIncomeLoss                        409
OperatingExpenses                   2867
ProfitLoss                          2429
PropertyPlantAndEquipmentGross       856
ResearchAndDevelopmentExpense       2516
StockholdersEquity                   241
dtype: int64


*Imputation*

Fill the missing values with mean value of the corresponding sector and industry. If the value is still blank after this then take the mean value of the corresponding sector.

In [127]:
# Fill missing values with the mean of the corresponding sector and industry
for column in transposed_df.columns:
    if transposed_df[column].isnull().any():
        transposed_df[column] = transposed_df.groupby(['Sector', 'Industry'])[column].transform(lambda x: x.fillna(x.mean()))

# If still missing, fill with the mean of the corresponding sector
for column in transposed_df.columns:
    if transposed_df[column].isnull().any():
        transposed_df[column] = transposed_df.groupby(['Sector'])[column].transform(lambda x: x.fillna(x.mean()))


# Check for missing values
missing_values = transposed_df.isnull().sum()

# Display missing values for each column
print(missing_values)

Concept
cik                                 0
entityName                          0
Fiscal Year                         0
Sector                              0
Industry                            0
Assets                              0
CostsAndExpenses                    0
GrossProfit                         0
IncomeTaxExpenseBenefit             0
Liabilities                         0
LiabilitiesAndStockholdersEquity    0
NetIncomeLoss                       0
OperatingExpenses                   0
ProfitLoss                          0
PropertyPlantAndEquipmentGross      0
ResearchAndDevelopmentExpense       0
StockholdersEquity                  0
dtype: int64


In [128]:
from google.colab import files
transposed_df.to_excel('transposed_df_v2.xlsx', index=False)
files.download('transposed_df_v2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>